In [1]:
import os
os.environ['PYTHONHASHSEED'] = '0'

import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(12345)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

# Rest of code follows ...

Using TensorFlow backend.


# Experiment 1: Six different depths of models, trained on 5 of the 6 subjects then evaluated on the 6th. 

*   0 degree aspect angle
*   Models:
    *  2-4
    *  4-8
    *  8-16
    *  16-32
    *  32-64
    *  64-128
  




## Notebook setup

Allow editing of modules using editor (auto reloading)

In [2]:
# Needed to allow editing using PyCharm
%load_ext autoreload
%autoreload 2

Needed for compatibility when using both CoLab and Local Jupyter notebook. It sets the appropriate file path for the data and also installs local packages such as models and data_loading.

In [3]:
# import os
if os.getcwd() == '/content':
    from google.colab import drive
    drive.mount('/content/gdrive')
    FILE_PATH = '/content/gdrive/My Drive/Level-4-Project/data/'
    !cd gdrive/My\ Drive/Level-4-Project/ && pip install --editable .
    os.chdir('gdrive/My Drive/Level-4-Project/')
    
else:
    FILE_PATH = "C:/Users/macka/Google Drive/Level-4-Project/data/"
    
# from src.models.original_models import cnn_various_depths
# from src.data import load_data

Import remaining packages

In [4]:
# import numpy as np
import sys
from six.moves import cPickle
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.utils import np_utils
import sys
from sklearn.metrics import classification_report, confusion_matrix
import csv
from keras.models import load_model

In [5]:
# Needed as originally code was for theano backend but now using tensor flow
# from keras import backend as K
K.set_image_dim_ordering('th')

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D


def make_model(depth, img_rows, img_cols, nb_classes):
    model = Sequential(name=depth)
    if depth == "2-4":
        model.add(Convolution2D(2, (3, 3), padding='same', input_shape=(1, img_rows, img_cols), activation='relu'))
        model.add(Convolution2D(2, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Convolution2D(4, (3, 3), padding='same', activation='relu'))
        model.add(Convolution2D(4, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

    elif depth == "4-8":
        model.add(Convolution2D(4, (3, 3), padding='same', input_shape=(1, img_rows, img_cols), activation='relu'))
        model.add(Convolution2D(4, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Convolution2D(8, (3, 3), padding='same', activation='relu'))
        model.add(Convolution2D(8, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

    elif depth == "8-16":
        model.add(Convolution2D(8, (3, 3), padding='same', input_shape=(1, img_rows, img_cols), activation='relu'))
        model.add(Convolution2D(8, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Convolution2D(16, (3, 3), padding='same', activation='relu'))
        model.add(Convolution2D(16, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

    elif depth == "16-32":
        model.add(Convolution2D(16, (3, 3), padding='same', input_shape=(1, img_rows, img_cols), activation='relu'))
        model.add(Convolution2D(16, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Convolution2D(32, (3, 3), padding='same', activation='relu'))
        model.add(Convolution2D(32, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

    elif depth == "32-64":
        model.add(Convolution2D(32, (3, 3), padding='same', input_shape=(1, img_rows, img_cols), activation='relu'))
        model.add(Convolution2D(32, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Convolution2D(64, (3, 3), padding='same', activation='relu'))
        model.add(Convolution2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

    elif depth == "64-128":
        model.add(Convolution2D(64, (3, 3), padding='same', input_shape=(1, img_rows, img_cols), activation='relu'))
        model.add(Convolution2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Convolution2D(128, (3, 3), padding='same', activation='relu'))
        model.add(Convolution2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    return model

In [7]:
import sys
from six.moves import cPickle

def load_batch(fpath, label_key='labels'):
    f = open(fpath, 'rb')
    if sys.version_info < (3,):
        d = cPickle.load(f)
    else:
        d = cPickle.load(f, encoding="bytes")
        # decode utf8
        for k, v in d.items():
            # added check as otherwise tries to decode a string
            if type(k) is not str:
                del(d[k])
                d[k.decode("utf8")] = v
    f.close()
    data = d["data"]
    labels = d[label_key]

    data = data.reshape(data.shape[0], 1, 75, 75)
    return data, labels


def load_data(path, nb_train_samples):
    X_train = np.zeros((nb_train_samples, 1, 75, 75), dtype="uint8")
    y_train = np.zeros((nb_train_samples,), dtype="uint8")
    offset = int(nb_train_samples/5)
    for i in range(1, 6):
        fpath = os.path.join(path, 'data_batch_' + str(i))
        data, labels = load_batch(fpath)
        X_train[(i-1)*offset:i*offset, :, :, :] = data
        y_train[(i-1)*offset:i*offset] = labels

    fpath = os.path.join(path, 'test_batch')
    X_test, y_test = load_batch(fpath)

    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))

    return (X_train, y_train), (X_test, y_test)


## Experiment Setup and Parameter Definition

In [8]:
batch_size = 100
nb_classes = 7
nb_epoch = 20
nb_epoch = 1
data_augmentation = False
nb_train_samples = 35595

# input image dimensions
img_rows, img_cols = 75, 75
# the CIFAR10 images are RGB


# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = load_data((FILE_PATH + 'cifar_initialised'), nb_train_samples)

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255 

models = {"2-4": {}, "4-8": {}, "8-16": {}, "16-32": {}, "32-64": {}, "64-128": {}}
for name, model in models.items():
    model["model"] = make_model(name, img_rows, img_cols, nb_classes)


## Training and Evaluating Models

In [ ]:
for name, model_data in models.items():
    # let's train the model using SGD + momentum (how original).
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model_data["model"].compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    if not data_augmentation:
        print('Not using data augmentation.')
        history = model_data["model"].fit(
            X_train,
            Y_train,
            batch_size=batch_size,
            epochs=nb_epoch,
            validation_data=(X_test, Y_test),
            shuffle=True, 
            verbose=1)

        y_pred = model_data["model"].predict_classes(X_test)
        target_names = ['ArmFasterTowards', 'ArmSlowerTowards', 'CirclingArm', 'Clapping', 'PickingUp', 'Sitting', 'Walking']
        model_data["history"] = history
        model_data["classification_report"] = classification_report(np.argmax(Y_test,axis=1), y_pred,target_names=target_names)
        model_data["confusion_matrix"] = confusion_matrix(np.argmax(Y_test,axis=1), y_pred)
#         model.save('/home/aleksandar/sets/cifar_initialised/results/model_initialised.h5')

    else:
        print('Using real-time data augmentation.')

        # this will do preprocessing and realtime data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images

        # compute quantities required for featurewise normalization
        # (std, mean, and principal components if ZCA whitening is applied)
        datagen.fit(X_train)

        # fit the model on the batches generated by datagen.flow()
        hist = model.fit_generator(datagen.flow(X_train, Y_train,
                            batch_size=batch_size),
                            samples_per_epoch=X_train.shape[0],
                            nb_epoch=nb_epoch,
                            validation_data=(X_test, Y_test))

        #NEW--------------------
        y_pred = model.predict_classes(X_test)
        print(y_pred)
        target_names = ['ArmFasterTowards', 'ArmSlowerTowards', 'CirclingArm', 'Clapping', 'PickingUp', 'Sitting', 'Walking']
#         sys.stdout = open('/home/aleksandar/sets/cifar_initialised/results/report_initialised.txt', "w")
        print("CLASSIFICATION REPORT:")
        print(classification_report(np.argmax(Y_test,axis=1), y_pred,target_names=target_names))
        print ("\n")
        print("CONFUSION MATRIX:")
        print(confusion_matrix(np.argmax(Y_test,axis=1), y_pred))

#         with open('/home/aleksandar/sets/cifar_initialised/results/graphs_initialised.csv', "w") as f:
#            w = csv.writer(f)
#            train_loss = hist.history['loss']
#            val_loss = hist.history['val_loss']
#            train_acc = hist.history['acc']
#            val_acc = hist.history['val_acc']
#            helplist = list(zip(train_loss, train_acc, val_loss, val_acc))
#            w.writerow("train_loss, train_acc, val_loss, val_acc")
#            for val in helplist:
#               w.writerow([val])


#         model.save('/home/aleksandar/sets/cifar_initialised/results/model_initialised.h5')
#         #-----------------------

Not using data augmentation.
Train on 35595 samples, validate on 7119 samples
Epoch 1/1
34900/35595 [============================>.] - ETA: 0s - loss: 0.6652 - acc: 0.7398

## Analysis

In [ ]:
import matplotlib.pyplot as plt

for name, model_data in models.items():
    plt.plot(model_data["history"].history['val_acc'], label=name)
    
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Train Accuracy')
plt.title('Experiment 1 (Figure 3a)')
plt.show()